... so we should expand the term in $\Gamma(\alpha/K)$, on the right hand side:

$$
\to_{K \to \infty}
\frac
  {K!}
  {\prod_{h=0}^{2^N-1}K_h!}
\left(\frac{\alpha}{K}\right)^K
\prod_{k=1}^{K^+}
\left(
\frac
  {\Gamma(m_k)\Gamma(N-m_k+1}
  {\Gamma(N + 1)}
  \right)
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {(\alpha/K)\Gamma(N + 1)}
  \right)^{K_0}
$$

$$
=
\frac
  {K!}
  {\prod_{h=0}^{2^N-1}K_h!}
\left(\frac{\alpha}{K}\right)^K
\prod_{k=1}^{K^+}
\left(
\frac
  {\Gamma(m_k)\Gamma(N-m_k+1}
  {\Gamma(N + 1)}
  \right)
\left(
\frac{K}
  {\alpha}
\right)^{K_0}
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {\Gamma(N + 1)}
  \right)^{K_0}
$$

Gotcha number two, that got me: whilst a single term in $a + b/K$ will tend to $a$ as $K \to \infty$, the same term raised to a power of $K$, will not necessarily do so, and the infinite power of a term that is assymptotic to one must be handled together.

$$
\left(\frac{\alpha}{K} \right)^K \left( \frac{K}{\alpha} \right)^{K_0}
  = \left(\frac{\alpha}{K} \right)^{K^+}
$$

Giving:

$$
\to_{K \to \infty}
\alpha^{K^+}
\frac
  {K!}
  {K^{K^+}\prod_{h=0}^{2^N-1}K_h!}
\prod_{k=1}^{K^+}
\left(
\frac
  {\Gamma(m_k)\Gamma(N-m_k+1}
  {\Gamma(N + 1)}
  \right)
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {\Gamma(N + 1)}
  \right)^{K_0}
 $$

Looking at the term:

$$\prod_{h=0}^{2^N-1}K_h!$$

$K_h \le K^+$ for $h>0$.  However, $K_0$ is not finite, so we should split that out separately:

$$
\to_{K \to \infty}
\alpha^{K^+}
\frac
  {K!}
  {K^{K^+}K_0!\prod_{h=1}^{2^N-1}K_h!}
\prod_{k=1}^{K^+}
\left(
\frac
  {\Gamma(m_k)\Gamma(N-m_k+1}
  {\Gamma(N + 1)}
  \right)
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {\Gamma(N + 1)}
  \right)^{K_0}
$$

And we have:

$$\frac{K!}{K_0!K^{K^+}} \to 1$$

(from earlier).  So:

$$
\to_{K \to \infty}
\frac
  {\alpha^{K^+}}
  {\prod_{h=1}^{2^N-1}K_h!}
\prod_{k=1}^{K^+}
\left(
\frac
  {\Gamma(m_k)\Gamma(N-m_k+1}
  {\Gamma(N + 1)}
  \right)
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {\Gamma(N + 1)}
  \right)^{K_0}
$$


Now we need to deal with the right-hand term, the one raised to the power $K_0$.  Initially, I assumed this was assymptotic to $1$, from looking at the terms inside, which tend themselves to $1$.  However, after inspecting the answer in the tutorial, it looks like we need to handle the terms inside the brackets together with the infinite power of $K_0$.

